In [1]:
import simpy
import random
import numpy as np

In [2]:
seed = 978
random.seed(seed)

# Arrival mean                              --EXPONENTIAL
interarrival_mean = 6 

# Operator 1 mean and std for service time  --LOGNORMAL
m = 12 
s = 6
M = np.log(m**2/np.sqrt(m**2+s**2))
S = np.log((m**2+s**2)/m**2)

# Operator 2 service time range             --UNIFORM
service_range = [1,7]

# Voice recognition mean -                  --EXPONENTIAL
router_mean = 5

In [3]:
router_customer = []
opr_1_break_times = []
opr_2_break_times = []
shift_started = True
end = False

In [4]:
def service(env, opr):
    if opr==operator1:
        yield env.timeout(random.lognormvariate(M,S))
        
    elif opr==operator2:
        yield env.timeout(random.uniform(*service_range))

In [5]:
class Customer(object):
    def __init__(self, name, env):
        self.env = env
        self.name = name
        self.arrival_t = self.env.now
        self.action = env.process(self.call())
        
    
    def call(self):
        print('%s initiated a call at %g' % (self.name, self.env.now))
        # Voice recognition and routing
        router_customer.append(self.name)
        yield env.timeout(random.expovariate(1/router_mean))
        if random.uniform(0,1) < .3:
            self.operator = operator1
        else:
            self.operator = operator2
            
        router_customer.remove(self.name)
            
        # Voice recognition failure
        if random.uniform(0,1) < .1:
            print('%s hangs up the call at %g' % (self.name, self.env.now))
            return 
        
        print('%s is rooted to operator %d at %g' % (self.name,(self.operator==operator2)+1, self.env.now))
        
        with self.operator.request() as req:
        
            result = yield req | env.timeout(10)
            # Reneging for 10 mins
            if req not in result:
                print('%s is left the operator %d after 10 minutes at %g' % (self.name,
                                                               (self.operator==operator2)+1,
                                                               self.env.now))
                return 
            print('%s is assigned to the operator %d at %g' % (self.name,
                                                               (self.operator==operator2)+1,
                                                               self.env.now))
            yield self.env.process(service(env,self.operator))
            print('%s is done at %g' % (self.name, self.env.now))
        
       

In [6]:
def is_available(opr):
    return opr.count == 0

In [7]:
#Determining breaks
def break_number():
    return np.random.poisson(8, size=2)

In [8]:
def insertionSort(operator_list):
    for i in range(1,len(operator_list)):
        val = operator_list[i]
        pos = i
        while pos>0 and operator_list[pos-1]>val:
            operator_list[pos]=operator_list[pos-1]
            pos = pos-1
        operator_list[pos]=val
    print(operator_list)

In [9]:
#Break request times and sorting
def break_time(opr_1_break, opr_2_break):
    for k in range(opr_1_break):
        opr_1_break_times.append(random.uniform(0,480))
    for k in range(opr_2_break):
        opr_2_break_times.append(random.uniform(0,480))
    insertionSort(opr_1_break_times)
    insertionSort(opr_2_break_times)

In [10]:
def shift(env):
    if end == False:
        [opr_1_break, opr_2_break] = break_number()
        print(opr_1_break)
        print(opr_2_break)
        break_time(opr_1_break, opr_2_break)
        yield env.timeout(480)
        env.process(shift(env))
        print("the time %g" % env.now)
        opr_1_break_times.clear()
        opr_2_break_times.clear()

In [11]:
def customer_generator(env):
    global end
    start = True
    for i in range(1000):
        
        #Every 480 a new shift starts.
        if start:
            env.process(shift(env))
            start = False
            
        yield env.timeout(random.expovariate(1/interarrival_mean))
        
        # Voice recognition system limit
        if len(router_customer) < 100:
            customer = Customer('Customer %s' %(i+1), env)
    end = True

In [12]:
env = simpy.Environment()
operator1 = simpy.Resource(env, capacity = 1)
operator2 = simpy.Resource(env, capacity = 1)
env.process(customer_generator(env))
env.run() 

8
5
[44.63307524333047, 74.39519318760645, 209.12059240981336, 255.9664319411964, 292.4094909717117, 385.08427703724, 387.8161389706391, 397.1018255104171]
[34.07604646614267, 90.0478027519914, 226.6161423659573, 414.16006451794135, 434.4019150966053]
Customer 1 initiated a call at 26.5136
Customer 2 initiated a call at 28.3499
Customer 1 is rooted to operator 2 at 28.7736
Customer 1 is assigned to the operator 2 at 28.7736
Customer 1 is done at 34.8671
Customer 2 is rooted to operator 2 at 34.8741
Customer 2 is assigned to the operator 2 at 34.8741
Customer 2 is done at 37.2446
Customer 3 initiated a call at 38.6398
Customer 3 is rooted to operator 2 at 38.828
Customer 3 is assigned to the operator 2 at 38.828
Customer 3 is done at 41.6471
Customer 4 initiated a call at 43.065
Customer 4 is rooted to operator 2 at 44.079
Customer 4 is assigned to the operator 2 at 44.079
Customer 4 is done at 45.2055
Customer 5 initiated a call at 58.8415
Customer 5 is rooted to operator 2 at 59.3744


Customer 370 is assigned to the operator 2 at 2349.15
Customer 370 is done at 2353.85
Customer 372 initiated a call at 2355.28
Customer 372 is rooted to operator 2 at 2356.01
Customer 372 is assigned to the operator 2 at 2356.01
Customer 373 initiated a call at 2357.71
Customer 372 is done at 2357.81
Customer 374 initiated a call at 2358.67
Customer 371 hangs up the call at 2359.55
Customer 374 hangs up the call at 2360.49
Customer 375 initiated a call at 2360.66
Customer 376 initiated a call at 2361.37
Customer 375 hangs up the call at 2362.88
Customer 377 initiated a call at 2367.8
Customer 373 is rooted to operator 2 at 2369.69
Customer 373 is assigned to the operator 2 at 2369.69
Customer 376 is rooted to operator 2 at 2372.38
Customer 377 is rooted to operator 2 at 2373.36
Customer 373 is done at 2375.88
Customer 376 is assigned to the operator 2 at 2375.88
Customer 378 initiated a call at 2377.53
Customer 378 is rooted to operator 2 at 2377.65
Customer 376 is done at 2382.83
Cust

Customer 860 initiated a call at 5110.86
Customer 861 initiated a call at 5111.63
Customer 862 initiated a call at 5114.62
Customer 862 is rooted to operator 2 at 5114.94
Customer 862 is assigned to the operator 2 at 5114.94
Customer 860 hangs up the call at 5118.41
Customer 862 is done at 5120.8
Customer 863 initiated a call at 5123.85
Customer 864 initiated a call at 5123.96
Customer 864 is rooted to operator 1 at 5125.13
Customer 864 is assigned to the operator 1 at 5125.13
Customer 861 is rooted to operator 2 at 5125.29
Customer 861 is assigned to the operator 2 at 5125.29
Customer 863 is rooted to operator 2 at 5125.71
Customer 861 is done at 5126.82
Customer 863 is assigned to the operator 2 at 5126.82
Customer 863 is done at 5132.95
Customer 864 is done at 5134.01
Customer 865 initiated a call at 5143.89
Customer 865 is rooted to operator 1 at 5143.96
Customer 865 is assigned to the operator 1 at 5143.96
Customer 866 initiated a call at 5147.94
Customer 866 is rooted to operator